In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
import re
import pandas as pd
import joblib
import numpy as np
import xgboost as xgb

xgb.set_config(verbosity=1, use_rmm=True)

In [2]:

# dataset_dir = 'sentiment140'
dataset_dir = 'imdb'
# dataset_dir = 'coronaNLP'

n_gram = (1, 1)
# n_gram = (1, 2)
# n_gram = (2, 2)

# importing the processed dataframe
df = joblib.load(f'../dataframes/df_{dataset_dir}.pkl')

df.head()


,review,sentiment
0,one review mention watch 1 oz episod youll hoo...,1
1,wonder littl product br br film techniqu veri ...,1
2,thought wonder way spend time hot summer weeke...,1
3,basic famili littl boy jake think zombi closet...,0
4,petter mattei love time money visual stun film...,1


In [3]:


X = df.iloc[:, 0]


y = df.iloc[:, 1]

X, y


(0        one review mention watch 1 oz episod youll hoo...
 1        wonder littl product br br film techniqu veri ...
 2        thought wonder way spend time hot summer weeke...
 3        basic famili littl boy jake think zombi closet...
 4        petter mattei love time money visual stun film...
                                ...                        
 49995    thought movi right good job wasnt creativ orig...
 49996    bad plot bad dialogu bad act idiot direct anno...
 49997    cathol taught parochi elementari school nun ta...
 49998    im go disagre previous comment side maltin one...
 49999    one expect star trek movi high art fan expect ...
 Name: review, Length: 49582, dtype: object,
 0        1
 1        1
 2        1
 3        0
 4        1
         ..
 49995    1
 49996    0
 49997    0
 49998    0
 49999    0
 Name: sentiment, Length: 49582, dtype: int32)

In [4]:

tfidf = joblib.load(
    f"../vectors/vectorizer_{dataset_dir}_{n_gram}.pkl")
tfidf


TfidfVectorizer(max_features=143417,
                tokenizer=<bound method RegexpTokenizer.tokenize of RegexpTokenizer(pattern='[a-zA-Z0-9]+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)>)

In [5]:
tfidf.vocabulary_, tfidf.idf_


({'one': 89676,
  'review': 104285,
  'mention': 78872,
  'watch': 135527,
  '1': 89,
  'oz': 91872,
  'episod': 41245,
  'youll': 140543,
  'hook': 59466,
  'right': 104781,
  'exact': 42385,
  'happen': 55834,
  'mebr': 78200,
  'br': 18352,
  'first': 46418,
  'thing': 125047,
  'struck': 119934,
  'brutal': 19516,
  'unflinch': 131117,
  'scene': 108479,
  'violenc': 134079,
  'set': 111234,
  'word': 138762,
  'go': 52015,
  'trust': 128872,
  'show': 112886,
  'faint': 43748,
  'heart': 56823,
  'timid': 126349,
  'pull': 99969,
  'punch': 100041,
  'regard': 102935,
  'drug': 37848,
  'sex': 111413,
  'hardcor': 55969,
  'classic': 25735,
  'use': 132414,
  'wordbr': 138765,
  'call': 21054,
  'nicknam': 85974,
  'given': 51713,
  'oswald': 90782,
  'maximum': 77522,
  'secur': 109875,
  'state': 118394,
  'penitentari': 93829,
  'focus': 47288,
  'main': 75509,
  'emerald': 40238,
  'citi': 25509,
  'experiment': 42930,
  'section': 109864,
  'prison': 98677,
  'cell': 22998,
 

In [6]:
X = tfidf.transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

X_train.shape, y_train.shape


((39665, 141506), (39665,))

In [8]:
# creating our pipeline that will return an estimator
pipeline = Pipeline(
    [('clf', xgb.XGBClassifier(max_depth=50, use_label_encoder=False))])

# xg = xgb.XGBClassifier(use_label_encoder=False) 
# xg

In [9]:

parameters = {
    'clf__booster': ('gbtree', 'gblinear', 'dart'),
    'clf__eta': (0.1, 0,25, 0.4, 0.5),
    }

# params = {
#           'booster': ['gbtree', 'gblinear', 'dart'],
#           'max_depth': [3, 6, 10, 15],
#           'eta': [0.01, 0.1, 0.2, 0.3, 0.4, 0.5],
#           'subsample': np.arange(0.5, 1.0, 0.1),
#           'colsample_bytree': np.arange(0.5, 1.0, 0.1),
#           'colsample_bylevel': np.arange(0.5, 1.0, 0.1),
#           'n_estimators': [100, 250, 500, 750],
#           }

clf = GridSearchCV(pipeline, param_grid=parameters,
                   scoring='accuracy', cv=5, verbose=1)

# clf = RandomizedSearchCV(xg, param_distributions=params,
#                          scoring='accuracy',
#                          n_iter=25,
#                          verbose=1)
clf


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('clf',
                                        XGBClassifier(base_score=None,
                                                      booster=None,
                                                      colsample_bylevel=None,
                                                      colsample_bynode=None,
                                                      colsample_bytree=None,
                                                      enable_categorical=False,
                                                      gamma=None, gpu_id=None,
                                                      importance_type=None,
                                                      interaction_constraints=None,
                                                      learning_rate=None,
                                                      max_delta_step=None,
                                                      max_depth=None,
                         

In [10]:

clf = clf.fit(X_train, y_train)


Fitting 5 folds for each of 15 candidates, totalling 75 fits
[16:04:27] WARNING: D:\Build\xgboost\xgboost-1.5.1.git\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:05:09] WARNING: D:\Build\xgboost\xgboost-1.5.1.git\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:05:45] WARNING: D:\Build\xgboost\xgboost-1.5.1.git\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:06:21] WARNING: D:\Build\xgboost\xgboost-1.5.1.git\src\learner.cc:1115: Starting in

In [11]:


y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))



              precision    recall  f1-score   support

           0       0.87      0.85      0.86      4939
           1       0.86      0.88      0.87      4978

    accuracy                           0.86      9917
   macro avg       0.86      0.86      0.86      9917
weighted avg       0.86      0.86      0.86      9917



In [12]:


print("Best: %f using %s" % (clf.best_score_, 
    clf.best_params_))
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
params = clf.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


Best: 0.868196 using {'clf__booster': 'gblinear', 'clf__eta': 0.4}
0.836581 (0.003637) with: {'clf__booster': 'gbtree', 'clf__eta': 0.1}
0.498147 (0.000050) with: {'clf__booster': 'gbtree', 'clf__eta': 0}
0.456170 (0.102669) with: {'clf__booster': 'gbtree', 'clf__eta': 25}
0.859599 (0.002075) with: {'clf__booster': 'gbtree', 'clf__eta': 0.4}
0.856599 (0.001493) with: {'clf__booster': 'gbtree', 'clf__eta': 0.5}
0.863608 (0.003148) with: {'clf__booster': 'gblinear', 'clf__eta': 0.1}
0.498147 (0.000050) with: {'clf__booster': 'gblinear', 'clf__eta': 0}
0.498147 (0.000050) with: {'clf__booster': 'gblinear', 'clf__eta': 25}
0.868196 (0.000508) with: {'clf__booster': 'gblinear', 'clf__eta': 0.4}
0.867692 (0.001200) with: {'clf__booster': 'gblinear', 'clf__eta': 0.5}
0.836581 (0.003637) with: {'clf__booster': 'dart', 'clf__eta': 0.1}
0.498147 (0.000050) with: {'clf__booster': 'dart', 'clf__eta': 0}
0.456170 (0.102669) with: {'clf__booster': 'dart', 'clf__eta': 25}
0.859599 (0.002075) with: {'

In [13]:




from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test, y_pred))

acc = int(accuracy_score(y_test, y_pred)*100)



[[4205  734]
 [ 612 4366]]


In [14]:


test_tweet = "groceri store"
vector = tfidf.transform([test_tweet])

print(clf.predict(vector))



[1]


In [16]:



# exporting the pipeline
joblib.dump(clf.best_estimator_, f'../models/xgb_{dataset_dir}_{acc}_{n_gram}.pkl')




['../models/xgb_imdb_86_(1, 1).pkl']